## SageMaker Pipelines Step Decorator Introduction

In this example we build a sample pipeline to show the functionality of using the new @step decorator that makes lift and shift of Python code easier.
- [Step Decorator Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-step-decorator.html)

### Setup

In [ ]:
#!pip install -U sagemaker

In [ ]:
import sagemaker
from sagemaker.workflow.function_step import step
from sagemaker.workflow.parameters import ParameterString

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name

# can also have as one param if same instance type
instance_type_one = ParameterString(name="AddNumsInstanceType", default_value="ml.m5.xlarge")
instance_type_two = ParameterString(name="MultipleNumsInstanceType", default_value="ml.m5.2xlarge")

### Steps Setup
We can define our steps with the @step decorator. For this dummy exercise we just include two steps, one that adds number and the second step takes that result and multiplies that number by 2.

In [ ]:
# step one
@step(
    name = "add-numbers",
    instance_type = instance_type_one,
    keep_alive_period_in_seconds=300
)
def add_num(x: int, y: int) -> int:
    z = x + y
    return z

In [ ]:
# step two
@step(
    name = "multiply-numbers",
    instance_type = instance_type_two,
    keep_alive_period_in_seconds=300
)
def multiply_num(z: int) -> int:
    res = z ** 2
    return res

In [ ]:
# stitch together pipeline
from sagemaker.workflow.pipeline import Pipeline

res_one = add_num(3, 4)
res_two = multiply_num(res_one)

pipeline = Pipeline(
    name="math-pipeline",
    parameters=[
        instance_type_one,
        instance_type_two,
    ],
    steps=[
        res_two,
    ],
)

### Pipeline Execution

In [ ]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.describe()
execution.wait()

In [ ]:
execution.list_steps()